# Initialize functions

## CNN

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function11 to build and compile a CNN model
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


## Hand-gesture

In [4]:
#Hand-gesture recognition v1 / Dataset generation and NN-training

import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from dataaugmentation import augment_dataset
from evaluation import evaluate_model
from CNN import build_model

# Load dataset
def load_dataset(data_dir, image_size=(64, 64)):
    images, labels = [], []
    label_map = {gesture: idx for idx, gesture in enumerate(os.listdir(data_dir))}
    for gesture, idx in label_map.items():
        gesture_dir = os.path.join(data_dir, gesture)
        for img_name in os.listdir(gesture_dir):
            if img_name.endswith('.png'):  # Ensure it's a PNG file
                img_path = os.path.join(gesture_dir, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
                img = cv2.resize(img, image_size) / 255.0  # Resize and normalize
                images.append(img)
                labels.append(idx)
    return np.array(images), np.array(labels), label_map

# Load data
data_dir = "dataset"  # Update with your dataset path
X, y, label_map = load_dataset(data_dir)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Add channel for grayscale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Loaded {len(X)} images, {len(X_train)} for training, {len(X_test)} for testing.")


# Main function
def main():
    # Load dataset
    data_dir = "dataset"  # Update this path with your dataset location
    X, y, label_map = load_dataset(data_dir)
    print(f"Dataset loaded with {len(X)} images.")
    
    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Data augmentation
    X_train_aug, y_train_aug = augment_dataset(X_train, y_train)
    print(f"Dataset augmented: {len(X_train_aug)} training samples.")
    
    # Build and train model
    model = build_model(input_shape=X_train_aug.shape[1:], num_classes=len(label_map))
    model.fit(X_train_aug, y_train_aug, epochs=10, validation_data=(X_test, y_test))
    
    # Evaluate model
    evaluate_model(model, X_test, y_test, label_map)

# Entry point
if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset'

## Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Function to evaluate and visualize results
def evaluate_model(model, X_test, y_test, label_map):
    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print(f"Accuracy: {accuracy}")
    sns.heatmap(conf_matrix, annot=True, cmap="Blues", xticklabels=label_map.keys(), yticklabels=label_map.keys())
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# Train CNN with augmented data

# Evaluate Model